In [4]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import io
import zipfile
import os
import sys
from tqdm.notebook import tqdm, trange

In [38]:
load = pd.read_excel('GEFCom2014/GEFCom2014-E/GEFCom2014-E.xlsx', parse_dates=['Date'])

In [39]:
load

,Date,Hour,load,T
0,2004-01-01,1,NaN,37.333333
1,2004-01-01,2,NaN,37.666667
2,2004-01-01,3,NaN,37.000000
3,2004-01-01,4,NaN,36.333333
4,2004-01-01,5,NaN,36.000000
...,...,...,...,...
96427,2014-12-31,20,4012.0,18.000000
96428,2014-12-31,21,3856.0,16.666667
96429,2014-12-31,22,3671.0,17.000000
96430,2014-12-31,23,3499.0,15.333333


In [41]:
load['timestamp'] = load['Date'].add(pd.to_timedelta(load.Hour - 1, unit='h'))

In [43]:
load.index = pd.to_datetime(load.timestamp)

In [50]:
load['2006-01':]

,Date,Hour,load,T,timestamp
timestamp,,,,,
2006-01-01 00:00:00,2006-01-01,1,3010.0,22.666667,2006-01-01 00:00:00
2006-01-01 01:00:00,2006-01-01,2,2853.0,20.666667,2006-01-01 01:00:00
2006-01-01 02:00:00,2006-01-01,3,2758.0,21.333333,2006-01-01 02:00:00
2006-01-01 03:00:00,2006-01-01,4,2705.0,19.000000,2006-01-01 03:00:00
2006-01-01 04:00:00,2006-01-01,5,2709.0,19.333333,2006-01-01 04:00:00
...,...,...,...,...,...
2014-12-31 19:00:00,2014-12-31,20,4012.0,18.000000,2014-12-31 19:00:00
2014-12-31 20:00:00,2014-12-31,21,3856.0,16.666667,2014-12-31 20:00:00
2014-12-31 21:00:00,2014-12-31,22,3671.0,17.000000,2014-12-31 21:00:00


In [9]:
# load forecasting competition data. Data is saved in energy.csv
def extract_data(data_dir):
    GEFCom_dir = os.path.join(data_dir, 'GEFCom2014', 'GEFCom2014 Data')

    GEFCom_zipfile = os.path.join(data_dir, 'GEFCom2014.zip')
    if not os.path.exists(GEFCom_zipfile):
        sys.exit("Download GEFCom2014.zip from https://www.dropbox.com/s/pqenrr2mcvl0hk9/GEFCom2014.zip?dl=0 and save it to the '{}' directory.".format(data_dir))

    # unzip root directory
    zip_ref = zipfile.ZipFile(GEFCom_zipfile, 'r')
    zip_ref.extractall(os.path.join(data_dir, 'GEFCom2014'))
    zip_ref.close()

    # extract the extended competition data
    zip_ref = zipfile.ZipFile(os.path.join(GEFCom_dir, 'GEFCom2014-E_V2.zip'), 'r')
    zip_ref.extractall(os.path.join(data_dir, 'GEFCom2014-E'))
    zip_ref.close()

    # load the data from Excel file
    data = pd.read_excel(os.path.join(data_dir, 'GEFCom2014-E', 'GEFCom2014-E.xlsx'), parse_dates=['Date'])

    # create timestamp variable from Date and Hour
    data['timestamp'] = data['Date'].add(pd.to_timedelta(data.Hour - 1, unit='h'))
    data = data[['timestamp', 'load', 'T']]
    data = data.rename(columns={'T':'temp'})

    # remove time period with no load data
    data = data[data.timestamp >= '2012-01-01']

    # save to csv
    data.to_csv(os.path.join(data_dir, 'energy.csv'), index=False)

In [10]:
extract_data('./')

In [69]:
FREQ = 'H'
DATETIME = 'datetime'
def load_raw_dataset():
    """
    Load the dataset as is.
    :return: pandas.DataFrame: sorted dataframe with parsed datetime
    """
    print('Hello')
    data_dir = 'GEFCom2014/Load/Task 1/'
    df = pd.read_csv(os.path.join(data_dir, 'L1-train.csv'))
    for i in range(2, 16):
        data_dir = 'GEFCom2014/Load/Task {}/'.format(i)
        tmp = pd.read_csv(os.path.join(data_dir, 'L{}-train.csv'.format(i)))
        df = pd.concat([df, tmp], axis=0)
    df[DATETIME] = pd.date_range('01-01-2001', '12-01-2011', freq=FREQ)[1:]
    df = df[~pd.isnull(df.LOAD)].reset_index(drop=True)
    return df

In [71]:
gefc = load_raw_dataset()

Hello


In [74]:
gefc

,ZONEID,TIMESTAMP,LOAD,w1,w2,w3,w4,w5,w6,w7,...,w17,w18,w19,w20,w21,w22,w23,w24,w25,datetime
0,1,112005 1:00,125.8,43,46,40,47,48,46,44,...,39,50,45,50,47,41,47,47,45,2005-01-01 01:00:00
1,1,112005 2:00,121.8,41,46,38,46,48,45,51,...,41,49,46,47,47,45,47,47,45,2005-01-01 02:00:00
2,1,112005 3:00,117.0,40,46,37,45,45,45,49,...,43,50,45,45,46,45,46,46,43,2005-01-01 03:00:00
3,1,112005 4:00,114.4,39,46,37,47,48,48,45,...,43,49,43,45,44,45,47,47,44,2005-01-01 04:00:00
4,1,112005 5:00,113.6,38,46,37,44,48,49,43,...,43,49,43,43,45,45,47,46,44,2005-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60595,1,11302011 20:00,183.3,38,34,34,42,41,42,41,...,34,46,39,41,48,37,41,43,37,2011-11-30 20:00:00
60596,1,11302011 21:00,183.3,36,34,34,41,39,40,38,...,34,46,39,39,46,37,40,44,36,2011-11-30 21:00:00
60597,1,11302011 22:00,178.3,35,36,34,39,39,38,38,...,32,46,37,37,46,36,38,42,42,2011-11-30 22:00:00
60598,1,11302011 23:00,166.4,33,37,32,38,39,39,37,...,30,45,37,36,46,36,37,41,41,2011-11-30 23:00:00


In [89]:
gefc = gefc.loc[:,['LOAD','datetime']]

In [90]:
gefc.index = gefc.datetime

In [103]:
gefc = gefc.drop(axis=1,columns='datetime')

In [104]:
gefc.to_csv('gefcom2014.csv')